# Задача 9. Hand-crafted graph features

- [x] Найти или сгенерировать набор данных для бинарной классификации графов.
- [x] Реализовать функцию `shortest_path_kernel(train_graphs, test_graphs)`, которая принимает тренировочный и тестовые наборы, а возвращает пару `K_train, K_test`
  - Опишите графы с помощью вектора из количества кратчайших путей различной длины
  - Для вычисления длин кратчайших путей можно использовать `nx.shortest_path_length(G)`
  - Ядровая функция для сравнения двух графов - скалярное произведение их двух векторов
  - `K_train` - матрица из ядровых функций для сравнения тренировочных графов между собой
  - `K_test` - матрица из ядровых функций для сравнения тестовых графов с тренировочными
- [x] Используя реализованное ядро обучите модель SVC, подберите гиперпараметры, вычислите различные метрики качества
- [x] (+5 баллов) Также реализовать Weisfeiler-Lehman Kernel и обучить классификатор с ним, сравнить результаты.

In [1]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset("data/", name="PROTEINS", use_node_attr=False, use_edge_attr=False)
dataset.download()

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from torch_geometric.utils import to_networkx


labels = np.array([data.y.item() for data in dataset])
indices = list(range(len(dataset)))
train_indices, test_indices, _, _ = train_test_split(
    indices,
    labels,
    test_size=0.2,
    stratify=labels,
    random_state=42,
)

all_graphs = np.array([to_networkx(x) for x in dataset], dtype=object)
train_graphs = all_graphs[train_indices]
test_graphs = all_graphs[test_indices]
train_target = labels[train_indices]
test_target = labels[test_indices]

In [3]:
import numpy as np
import networkx as nx


def get_spls(graphs: list[nx.MultiDiGraph], lenghts_vector_length=10) -> np.ndarray:
    lenghts = np.zeros((len(graphs), lenghts_vector_length), dtype=int)
    SHIFT = 1
    for i, graph in enumerate(graphs):
        p = nx.shortest_path_length(graph)
        p = list(p)
        for node, spls in p:
            for node_neighbour, length in spls.items():
                if node_neighbour < node + 1:
                    continue
                if length > lenghts_vector_length:
                    lenghts[i,-1] += 1
                else:
                    lenghts[i, length - SHIFT] += 1
    return lenghts

def graph_kernel_sp(train_graphs, test_graphs, lenghts_vector_length, seed=20):
    np.random.seed(seed)
    train_spls = get_spls(train_graphs, lenghts_vector_length)
    test_splts = get_spls(test_graphs, lenghts_vector_length)

    K_train = np.dot(train_spls, train_spls.T)
    K_test = np.dot(test_splts, train_spls.T)

    return K_train, K_test


In [4]:


K_train_gk, K_test_gk = graph_kernel_sp(
    train_graphs, test_graphs, lenghts_vector_length=15, seed=42
)

print(K_train_gk.shape, K_test_gk.shape)

(890, 890) (223, 890)


In [5]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

param_grid = {
    "C": [0.01, 0.03, 0.1],
    "gamma": [
        0.001,
        0.01,
        0.1
    ],
}
model = SVC(kernel="precomputed", probability=True, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print("Starting Hyperparameter Tuning")
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=cv,
    refit="accuracy",
    verbose=2,
    return_train_score=False,
)
grid_search.fit(K_train_gk, train_target)

print("Best parameters found:", grid_search.best_params_)
print("Best accuracy score:", grid_search.best_score_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(K_test_gk)

print("\nTest set performance:")
print("Accuracy:", accuracy_score(test_target, y_pred))
print("Precision:", precision_score(test_target, y_pred))
print("Recall:", recall_score(test_target, y_pred))
print("F1 score:", f1_score(test_target, y_pred))


Starting Hyperparameter Tuning
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ................................C=0.01, gamma=0.001; total time=   4.2s
[CV] END ................................C=0.01, gamma=0.001; total time= 1.1min
[CV] END ................................C=0.01, gamma=0.001; total time=   4.3s
[CV] END ................................C=0.01, gamma=0.001; total time=   2.1s
[CV] END ................................C=0.01, gamma=0.001; total time=   5.6s
[CV] END .................................C=0.01, gamma=0.01; total time=   4.4s
[CV] END .................................C=0.01, gamma=0.01; total time= 1.1min
[CV] END .................................C=0.01, gamma=0.01; total time=   4.2s
[CV] END .................................C=0.01, gamma=0.01; total time=   2.1s
[CV] END .................................C=0.01, gamma=0.01; total time=   5.6s
[CV] END ..................................C=0.01, gamma=0.1; total time=   4.2s
[CV] END .........

SPL посчитался за 44.42 минуты

In [ ]:
from typing import Optional

def get_wls_colors(
    graphs: list[nx.Graph],
    iters: Optional[int] = None,
) -> np.ndarray:
    max_nodes = 0
    for G in graphs:
        max_nodes = max(len(G.nodes), max_nodes)

    INIT_COLOR = 1
    colors_prev = np.ones((len(graphs), max_nodes), dtype=int)
    colors_next = np.ones((len(graphs), max_nodes), dtype=int)
    unique_colors_prev = np.zeros(len(graphs), dtype=int)
    unique_colors_next = np.zeros(len(graphs), dtype=int)

    aggregated: dict[tuple, int] = {(INIT_COLOR,): 1}
    iteration = 0
    while True:
        for i, G in enumerate(graphs):
            if unique_colors_next[i] == unique_colors_prev[i] and iteration > 0:
                continue
            unique_colors_prev[i] = unique_colors_next[i]
            for v in G.nodes:
                neighbours = G[v].keys()
                colors = [0] * len(neighbours)
                for j, neighbour in enumerate(neighbours):
                    colors[j] = int(colors_prev[i, neighbour])
                colors.sort()
                colors = tuple(colors)
                if colors not in aggregated:
                    aggregated[colors] = len(aggregated) + 1
                colors_next[i, v] = aggregated[colors]
            unique_colors_next[i] = np.unique_values(colors_next[i]).shape[0]
        if (unique_colors_next == unique_colors_prev).all():
            break
        iteration += 1
        if iters is not None and iteration != iters:
            break
        colors_prev = colors_next.copy()
    return colors_prev


def get_wls_colors_distribution(
    graphs: list[nx.Graph],
    max_colors: Optional[int] = None,
    iters: Optional[int] = None,
) -> np.ndarray:
    graphs_colors = get_wls_colors(graphs, iters)
    if max_colors is None:
        max_colors = np.unique_values(graphs_colors).shape[0]

    lenghts = np.zeros((len(graphs), max_colors), dtype=int)
    colors_map = dict()
    cur_color = 0
    for i in range(len(graphs)):
        unique, counts = np.unique(graphs_colors[i], return_counts=True)
        pairs = zip(unique, counts)
        for v, count in pairs:
            v = int(v)
            if v not in colors_map:
                colors_map[v] = cur_color
                color_index = cur_color
                cur_color += 1
            else:
                color_index = colors_map[v]
            if color_index < max_colors:
                lenghts[i, color_index] = count
    return lenghts


def graphlet_kernel_wl(
    graphs,
    train_indices,
    test_indices,
    max_colors: Optional[int] = None,
    iters: Optional[int] = None,
):
    wl_lengths = get_wls_colors_distribution(graphs, max_colors, iters)
    train_wl_lengths = wl_lengths[train_indices]
    test_wl_lengths = wl_lengths[test_indices]

    K_train = np.dot(train_wl_lengths, train_wl_lengths.T)
    K_test = np.dot(test_wl_lengths, train_wl_lengths.T)

    return K_train, K_test

Тест для проверки:

In [5]:
edges = [
    (0, 1),
    (0, 2),
    (0, 3),
    (1, 3),
    (2, 3),
    (2, 4),
    (2, 5),
]
G = nx.Graph(edges)
edges = [
    (0, 1),
    (0, 2),
    (1, 3),
    (1, 2),
    (2, 3),
    (2, 4),
    (3, 5),
]
G2 = nx.Graph(edges)
K_train = get_wls_colors([G, G2])
K_train_gk = get_wls_colors_distribution([G, G2])
display(K_train)
display(K_train_gk)

array([[ 5,  6,  7,  5,  8,  8],
       [ 9,  5, 10, 11,  8, 12]])

array([[2, 1, 1, 2, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1]])

In [6]:
K_train_wl, K_test_wl = graphlet_kernel_wl(all_graphs, train_indices, test_indices)

print(K_train_wl.shape, K_test_wl.shape)


(890, 890) (223, 890)


In [15]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

param_grid = {
    "C": [0.01, 0.03, 0.1],
    "gamma": [0.001, 0.01, 0.1],
}

model = SVC(kernel="precomputed", probability=True, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print("Starting Hyperparameter Tuning")
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=cv,
    refit="accuracy",
    verbose=2,
    return_train_score=False,
)
grid_search.fit(K_train_wl, train_target)

print("Best parameters found:", grid_search.best_params_)
print("Best accuracy score:", grid_search.best_score_)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(K_test_wl)

print("\nTest set performance:")
print("Accuracy:", accuracy_score(test_target, y_pred))
print("Precision:", precision_score(test_target, y_pred))
print("Recall:", recall_score(test_target, y_pred))
print("F1 score:", f1_score(test_target, y_pred))


Starting Hyperparameter Tuning
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ................................C=0.01, gamma=0.001; total time=   0.0s
[CV] END ................................C=0.01, gamma=0.001; total time=   0.0s
[CV] END ................................C=0.01, gamma=0.001; total time=   0.0s
[CV] END ................................C=0.01, gamma=0.001; total time=   0.0s
[CV] END ................................C=0.01, gamma=0.001; total time=   0.0s
[CV] END .................................C=0.01, gamma=0.01; total time=   0.0s
[CV] END .................................C=0.01, gamma=0.01; total time=   0.0s
[CV] END .................................C=0.01, gamma=0.01; total time=   0.0s
[CV] END .................................C=0.01, gamma=0.01; total time=   0.0s
[CV] END .................................C=0.01, gamma=0.01; total time=   0.0s
[CV] END ..................................C=0.01, gamma=0.1; total time=   0.0s
[CV] END .........

Итог: 


| Method | Time (s) | Accuracy | Precision | Recall | F1 score |
| ------ | -------- | -------- | --------- | ------ | -------- | 
| SP     |   2665   | 0.72     | 0.65      | 0.65   |   0.65   |
| WL     |   3.6   | 0.69     | 0.67      | 0.43   | 0.53     |




WL Kernel показал сопоставимые результаты с SPL на тестовой выборке (0.53 против 0.65 F1 score), однако он оказался вычислительно гораздо эффективнее (почти в 1000 раз). 